# Poses Submission for FEGrow/ A3FE

In [1]:
import polaris as po
import base64
from tqdm import tqdm
from rdkit import Chem
import datamol as dm
from pathlib import Path

competition = po.load_competition("asap-discovery/antiviral-ligand-poses-2025")
competition.cache()
_, test = competition.get_train_test_split()

[2025-02-25 18:25:50] INFO     The version of Polaris that was used to create the artifact (0.11.0) ]8;id=914781;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=483354;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/_artifact.py#92\92]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.11.8).                                                                           

                      WARNING  You're loading data from a remote location. If the dataset is small     ]8;id=76498;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=458395;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/dataset/_base.py#181\181]8;;\
                               enough, consider caching the dataset first using                                    
                               CompetitionSpecification.cache() for more performant data access.                   

Output()

[18:25:51] Copying to destination                                                                      ]8;id=453665;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=105581;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/dataset/_base.py#380\380]8;;\
           /home/finlayclark/.cache/polaris/datasets/8f1eaa9c-c80c-4e94-b39c-3427ff26cf2a/data.zarr                

           For large Zarr archives, this may take a while.                                             ]8;id=743254;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=31477;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/dataset/_base.py#381\381]8;;\

[18:26:01]  Success: Copying Zarr keys                                                                ]8;id=769216;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=416298;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

            Success: Copying Zarr archive                                                             ]8;id=969284;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=984927;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\

In [2]:
def get_abfe_runs_dict(abfe_dir: Path) -> dict[int, [int, Path]]:
    """Create a dictionary of {ligand_index: {pose_index: Path}} from the ABFE directory."""
    lig_idxs = set([int(calc_dir.name.split("_")[1]) for calc_dir in abfe_dir.glob("lig_*")])
    lig_runs = {}
    for lig_idx in lig_idxs:
        lig_runs[lig_idx] = {int(calc_dir.name.split("_")[2]): calc_dir for calc_dir in abfe_dir.glob(f"lig_{lig_idx}_*")}
    return lig_runs

# Pure docking
MERS_DOCKED_SDF = Path("../mers-gnina/merged-MERS.sdf")
MERS_ALTERNATE_PROTONATION = Path("../mers-gnina/cs_optimised_reprotonated.sdf")
SARS_DOCKED_SDF = Path("../sars-gnina/merged-SARS.sdf")

# ABFE
MERS_ABFE_DIR = Path("../mers-040225")
# Store the SDFs as a dict of ligand_index: SDF_path
MERS_ABFE_SDFS = {int(lig_sdf.name.split("_")[-1].split(".")[0]): lig_sdf for lig_sdf in (MERS_ABFE_DIR / "input" / "full_run-MERS" / "structures").glob("*.sdf")}
MERS_ABFE_RUNS = get_abfe_runs_dict(MERS_ABFE_DIR)

SARS_ABFE_DIR = Path("../sars-290125-plato")
SARS_ABFE_SDFS = {int(lig_sdf.name.split("_")[-1].split(".")[0]): lig_sdf for lig_sdf in (SARS_ABFE_DIR / "input" / "full_run-SARS" / "structures").glob("*.sdf")}
SARS_ABFE_RUNS = get_abfe_runs_dict(SARS_ABFE_DIR)

SARS_ABFE_EXTRA_POSES_DIR = Path("../sars-extra-poses-210225")
SARS_ABFE_EXTRA_POSES_SDFS = {int(lig_sdf.name.split("_")[-1].split(".")[0]): lig_sdf for lig_sdf in (SARS_ABFE_EXTRA_POSES_DIR / "input" / "structures-SARS-comp48").glob("*.sdf")}
SARS_ABFE_EXTRA_POSES_RUNS = get_abfe_runs_dict(SARS_ABFE_EXTRA_POSES_DIR)

# Containers for results
y_pred_mers = []
y_pred_sars = []

## Extract Docked Structures

These are just the best-scoring (with GNINA) structures from docking (for problematic poses for which ABFE was not run).

In [3]:
def read_sdf(sdf:Path)-> list[Chem.Mol]:
    with Chem.SDMolSupplier(sdf.as_posix()) as suppl:
        mols = [mol for mol in suppl if mol]
        # Check that no molecules are None
        assert all(mols)
    return mols

y_pred_mers.extend(read_sdf(MERS_DOCKED_SDF))
y_pred_mers.extend(read_sdf(MERS_ALTERNATE_PROTONATION))
y_pred_sars.extend(read_sdf(SARS_DOCKED_SDF))

[18:26:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[18:26:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[18:26:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[18:26:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[18:26:01] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


## Select the Poses Scored Best with Fast ABFE

(Run using A3FE)

In [5]:
def read_abfe_score(abfe_dir: Path) -> float:
    lines = (abfe_dir / "output" / "overall_stats.dat").read_text().split("\n")
    return float(lines[1].split("Mean free energy: ")[1].split(" ")[0])

def extract_best_scored_conformers(sdfs: dict[int, Path], runs: dict[int, dict[int, Path]]) -> list[Chem.Mol]:
    """Get the best scored conformer for each ligand from the ABFE runs."""

    mols = []

    for mol_idx, sdf in tqdm(sdfs.items()):
        conformers = read_sdf(sdf)

        # If there is no ABFE run for this ligand, just take the first conformer
        if mol_idx not in runs.keys():
            mols.append(conformers[0])
            continue

        # Get the best scored conformer
        conformer_scores = {pose_idx: read_abfe_score(abfe_dir) for pose_idx, abfe_dir in runs[mol_idx].items()}
        best_pose_idx = min(conformer_scores.keys(), key=lambda x: conformer_scores[x])
        mols.append(conformers[best_pose_idx])

    return mols

y_pred_mers.extend(extract_best_scored_conformers(MERS_ABFE_SDFS, MERS_ABFE_RUNS))
y_pred_sars.extend(extract_best_scored_conformers(SARS_ABFE_SDFS, SARS_ABFE_RUNS))
y_pred_sars.extend(extract_best_scored_conformers(SARS_ABFE_EXTRA_POSES_SDFS, SARS_ABFE_EXTRA_POSES_RUNS))

100%|██████████| 3/3 [00:00<00:00, 537.02it/s]


## Format Submission

Merge the conformers into a list with the expected order.

In [6]:
y_pred_merged = []
protein_label_to_pred = {"MERS-CoV Mpro": y_pred_mers, "SARS-CoV-2 Mpro": y_pred_sars}

def remove_double_bond_stereo(smiles:str) -> str:
    mol = Chem.MolFromSmiles(smiles)
    for bond in mol.GetBonds():
        if bond.GetBondType() == Chem.BondType.DOUBLE:
            bond.SetStereo(Chem.BondStereo.STEREONONE)  # Remove E/Z info
    return Chem.CanonSmiles(Chem.MolToSmiles(mol, isomericSmiles=True))  # Keep chirality

for test_idx in tqdm(range(len(test))):
    found_match = False
    test_smiles = test[test_idx]["CXSMILES"]
    protein_label = test[test_idx]["Protein Label"]

    for pred_sdf in protein_label_to_pred[protein_label]:
        pred_smiles = Chem.CanonSmiles(Chem.MolToSmiles(pred_sdf))
        if test_smiles == pred_smiles:
            y_pred_merged.append(pred_sdf)
            found_match = True
            break

    # If we haven't found a match yet, try removing double bond stereochmistry
    if not found_match:
        for pred_sdf in protein_label_to_pred[protein_label]:
            pred_smiles = Chem.CanonSmiles(Chem.MolToSmiles(pred_sdf))
            pred_smiles = remove_double_bond_stereo(pred_smiles)
            if test_smiles == pred_smiles:
                y_pred_merged.append(pred_sdf)
                found_match = True
                break

    if not found_match:
        raise RuntimeError(f"No matching prediction found for test index {test_idx, protein_label, test_smiles}")

100%|██████████| 195/195 [00:04<00:00, 46.44it/s]


Sanity checking...

In [7]:
def assert_same_coordinates(ref_mol: Chem.Mol, pred_mol: Chem.Mol) -> None:
    # Check that we only have one conformer
    assert ref_mol.GetNumConformers() == 1
    assert pred_mol.GetNumConformers() == 1

    # Check that the SMILES are the same
    ref_smiles = Chem.CanonSmiles(Chem.MolToSmiles(ref_mol))
    pred_smiles = Chem.CanonSmiles(Chem.MolToSmiles(pred_mol))
    assert ref_smiles == pred_smiles, "Smiles are not the same"

    # Check taht the coordinates are the same
    ref_coords = ref_mol.GetConformer().GetPositions()
    pred_coords = pred_mol.GetConformer().GetPositions()
    assert ref_coords.shape == pred_coords.shape
    assert (ref_coords == pred_coords).all(), "Coordinates are not the same"


In [8]:
# Check that the coordinates of the first MERS ligand match the input for the best-scoring ABFE conformer
# The pose with idx 4 had the best ABFE score for ligand 0
ref_mol = read_sdf(MERS_ABFE_RUNS[0][4] / "input" / "ligand.sdf")[0]
pred_mol = y_pred_merged[0]
assert_same_coordinates(ref_mol, pred_mol)

# Check that the coordinates of the last MERS ligand match the input for the best-scoring ABFE conformer
# The pose with idx 3 had the best ABFE score for this ligand
ref_mol = read_sdf(MERS_ABFE_RUNS[96][3] / "input" / "ligand.sdf")[0]
pred_mol = y_pred_merged[-3]
assert_same_coordinates(ref_mol, pred_mol)

# Test one SARS ligand
ref_mol = read_sdf(SARS_ABFE_RUNS[1][0] / "input" / "ligand.sdf")[0]
pred_mol = y_pred_merged[5]
assert_same_coordinates(ref_mol, pred_mol)

In [9]:
# Write out one structure to sdf to visualise
with Chem.SDWriter("test.sdf") as writer:
    writer.write(y_pred_merged[-3])

In [10]:
def serialize_rdkit_mol(mol: Chem.Mol) -> str:
    props = Chem.PropertyPickleOptions.AllProps
    mol_bytes = mol.ToBinary(props)
    return base64.b64encode(mol_bytes).decode('ascii')  


y_pred_serialized = [serialize_rdkit_mol(mol) for mol in y_pred_merged]

## Submit

In [11]:
competition.submit_predictions(
    predictions=y_pred_serialized,
    prediction_name="newcastle-edinburgh-fegrow-a3fe",
    prediction_owner="fjclark",
    report_url="https://github.com/michellab/polaris-poses-challenge-fegrow-a3fe", 
    github_url="https://github.com/michellab/polaris-poses-challenge-fegrow-a3fe",
    description="FEgrow/A3FE submission by Finlay Clark, Asma Feriel Khoualdi, Josh Horton, Julien Michel and Daniel Cole (v1)",
    tags=["FEgrow", "ANI", "OpenMM", "ABFE", "A3FE", "MD"],
    user_attributes={"Framework": "FEgrow/A3FE", "Method": "Constrained geometry optimisation with ML/MM, followed by scoring with fast ABFE calculations."}
)

[18:26:12]  Success: Submitting competition predictions                                               ]8;id=582851;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=13390;file:///home/finlayclark/anaconda3/envs/mamba/envs/polaris-challenge-a3fe/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\